In [9]:
from datetime import datetime

In [10]:
datetime.now()

datetime.datetime(2025, 5, 21, 11, 18, 37, 236731)

In [14]:
pip install requests

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)

   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   -------- ------------------------------- 1/5 [idna]
   -------- ------------------------------- 1/5 [idna]
   -------- ------------------------------- 1/5 [idna]
   -------- ------------------------------- 1/5 [idna]
   ---------------- ----------------------- 2/5 [charset-normalizer]
   ---------------- ----------------------- 2/5 [charset-normalizer]
   -----

In [15]:
import requests

In [17]:
pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
    --------------------------------------- 0.3/12.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.6 MB 1.0 MB/s eta 0:00:12
   -- ------------------------------------- 0.8/12.6 MB 1.2 MB/s eta 0:00:10
   --- ------------------------------------ 1.0/12.6 MB 1.3 MB/s eta 0:00:10
   --- ------------------------------------ 1.0/12.6 MB 1.3 MB/s eta 0:00:10
   --- ------------------------------------ 1.0/12.6 MB 1.3 MB/s eta 0:00:10
   --- ------------------------------------ 1.0/12.6 MB 1.3 MB/s eta 0:00:10
   --- ------------------------------------ 1.0/12.6 MB 1.3 MB/s eta 0:00:10
   --- ------------------------------------ 1.0/12.6 MB 1.3 MB/s eta 0:00:10
   --- ----------------------------------

In [18]:
import pandas as pd


In [19]:
import csv

In [20]:
from io import StringIO

In [21]:
import os


In [22]:
from datetime import datetime


In [23]:

NIFTY_50_SYMBOLS = {
    'ADANIENT', 'ADANIPORTS', 'APOLLOHOSP', 'ASIANPAINT', 'AXISBANK',
    'BAJAJ-AUTO', 'BAJFINANCE', 'BAJAJFINSV', 'BPCL', 'BHARTIARTL',
    'BRITANNIA', 'CIPLA', 'COALINDIA', 'DIVISLAB', 'DRREDDY',
    'EICHERMOT', 'GRASIM', 'HCLTECH', 'HDFCBANK', 'HDFCLIFE',
    'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'ICICIBANK', 'ITC',
    'INDUSINDBK', 'INFY', 'JSWSTEEL', 'KOTAKBANK', 'LT',
    'M&M', 'MARUTI', 'NTPC', 'NESTLEIND', 'ONGC',
    'POWERGRID', 'RELIANCE', 'SBILIFE', 'SBIN', 'SUNPHARMA',
    'TCS', 'TATACONSUM', 'TATAMOTORS', 'TATASTEEL', 'TECHM',
    'TITAN', 'ULTRACEMCO', 'WIPRO', 'APOLLOHOSP', 'ADANIPORTS'
}

In [24]:
def download_nse_master_file():
    """
    Download the NSE Master file from Kite (Zerodha)
    """
    url = "https://api.kite.trade/instruments"
    
    try:
        print("Downloading NSE Master file from Kite...")
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        
        with open('nse_master_raw.csv', 'w', encoding='utf-8') as f:
            f.write(response.text)
        
        print("✓ NSE Master file downloaded successfully")
        return response.text
    
    except requests.exceptions.RequestException as e:
        print(f"✗ Error downloading file: {e}")
        raise

In [25]:
def parse_master_file(file_content):
    """
    Parse the NSE Master file content
    """
    try:
        print("Parsing NSE Master file...")
        
        df = pd.read_csv(StringIO(file_content))
        
        print(f"✓ Total instruments found: {len(df)}")
        print(f"✓ Columns: {list(df.columns)}")
        
        print("\nFirst 5 rows of the master file:")
        print(df.head())
        
        return df
    
    except Exception as e:
        print(f"✗ Error parsing file: {e}")
        raise

In [26]:
def filter_nifty50_stocks(df):
    """
    Filter out NIFTY 50 stocks from the master file
    """
    try:
        print("\nFiltering NIFTY 50 stocks...")
        
        nse_equity = df[
            (df['exchange'] == 'NSE') & 
            (df['segment'] == 'NSE') &
            (df['instrument_type'] == 'EQ')
        ]
        
        print(f"✓ NSE Equity instruments: {len(nse_equity)}")
        
        nifty50_stocks = nse_equity[
            nse_equity['tradingsymbol'].isin(NIFTY_50_SYMBOLS)
        ]
        
        print(f"✓ NIFTY 50 stocks found: {len(nifty50_stocks)}")
        
        if len(nifty50_stocks) > 0:
            print("\nNIFTY 50 stocks found:")
            for symbol in sorted(nifty50_stocks['tradingsymbol'].values):
                print(f"  - {symbol}")
        
        return nifty50_stocks
    
    except Exception as e:
        print(f"✗ Error filtering stocks: {e}")
        raise

In [27]:
def save_to_csv(df, filename):
    """
    Save the filtered NIFTY 50 stocks to a CSV file
    """
    try:
        print(f"\nSaving NIFTY 50 stocks to {filename}...")
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename_with_timestamp = f"nifty50_stocks_{timestamp}.csv"
        
        df.to_csv(filename_with_timestamp, index=False)
        
        df.to_csv(filename, index=False)
        
        print(f"✓ Files saved successfully:")
        print(f"  - {filename}")
        print(f"  - {filename_with_timestamp}")
        
        print(f"\nSummary:")
        print(f"  - Total NIFTY 50 stocks saved: {len(df)}")
        print(f"  - File size: {os.path.getsize(filename)} bytes")
        
    except Exception as e:
        print(f"✗ Error saving file: {e}")
        raise

In [28]:
def display_missing_stocks(found_stocks):
    """
    Display any NIFTY 50 stocks that were not found in the master file
    """
    found_symbols = set(found_stocks['tradingsymbol'].values)
    missing_symbols = NIFTY_50_SYMBOLS - found_symbols
    
    if missing_symbols:
        print(f"\n⚠ Warning: {len(missing_symbols)} NIFTY 50 stocks not found:")
        for symbol in sorted(missing_symbols):
            print(f"  - {symbol}")
    else:
        print("\n✓ All NIFTY 50 stocks found successfully!")

In [29]:
def main():
    """
    Main function to orchestrate the entire process
    """
    print("=" * 60)
    print("NSE Master File Parser for NIFTY 50 Stocks")
    print("=" * 60)
    
    try:
        file_content = download_nse_master_file()
        
        master_df = parse_master_file(file_content)
        
        nifty50_df = filter_nifty50_stocks(master_df)
        
        display_missing_stocks(nifty50_df)
        
        if len(nifty50_df) > 0:
            save_to_csv(nifty50_df, 'nifty50_stocks.csv')
        else:
            print("\n⚠ No NIFTY 50 stocks found to save!")
        
        print("\n" + "=" * 60)
        print("Process completed successfully!")
        print("=" * 60)
        
    except Exception as e:
        print(f"\n✗ Process failed: {e}")
        return False
    
    return True


In [30]:
def analyze_nifty50_data(csv_file='nifty50_instruments.csv'):
    """
    Analyze the saved NIFTY 50 data
    """
    try:
        df = pd.read_csv(csv_file)
        
        print(f"\nAnalysis of NIFTY 50 data:")
        print(f"  - Total stocks: {len(df)}")
        print(f"  - Columns: {list(df.columns)}")
        
        if 'name' in df.columns:
            print(f"\nStock names:")
            for _, row in df.iterrows():
                print(f"  {row['tradingsymbol']:12} - {row['name']}")
        
        return df
    
    except FileNotFoundError:
        print(f"✗ File {csv_file} not found. Run the main process first.")
        return None
    except Exception as e:
        print(f"✗ Error analyzing data: {e}")
        return None

In [31]:
def update_nifty50_list():
    """
    Function to update the NIFTY 50 list if needed
    Note: NIFTY 50 composition changes periodically
    """
    print("Note: NIFTY 50 composition may change over time.")
    print("Current list is based on October 2024 composition.")
    print("For the most current list, please check NSE official website.")

if __name__ == "__main__":
    success = main()
    
    if success:
        print("\nWould you like to analyze the saved data? (y/n)")
        user_input = input().strip().lower()
        if user_input == 'y':
            analyze_nifty50_data()

NSE Master File Parser for NIFTY 50 Stocks
✓ NSE Master file downloaded successfully
Parsing NSE Master file...
✓ Total instruments found: 94355
✓ Columns: ['instrument_token', 'exchange_token', 'tradingsymbol', 'name', 'last_price', 'expiry', 'strike', 'tick_size', 'lot_size', 'instrument_type', 'segment', 'exchange']

First 5 rows of the master file:
   instrument_token  exchange_token   tradingsymbol    name  last_price  \
0         223969029          874879  BANKEX25MAYFUT  BANKEX           0   
1         281854981         1100996  BANKEX25JUNFUT  BANKEX           0   
2         292126213         1141118  BANKEX25JULFUT  BANKEX           0   
3         223938309          874759  SENSEX25MAYFUT  SENSEX           0   
4         287893253         1124583  SENSEX25603FUT  SENSEX           0   

       expiry  strike  tick_size  lot_size instrument_type  segment exchange  
0  2025-05-27     0.0       0.05        30             FUT  BFO-FUT      BFO  
1  2025-06-24     0.0       0.05    